In [0]:
%pip install dbldatagen

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
%pip install Faker

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
from dbldatagen import DataGenerator, PyfuncText, DateRange
from faker import Faker
import random
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType, DoubleType

# Create a SparkSession
spark = SparkSession.builder.appName("DataGeneration").getOrCreate()

brand_values = ["Brand1", "Brand2", "Brand3", "Brand4", "Brand5"]
sub_brand_values = ["SubBrand1", "SubBrand2", "SubBrand3", "SubBrand4", "SubBrand5"]
category_values = ["Category1", "Category2", "Category3", "Category4", "Category5"]
sub_category_values = ["SubCategory1", "SubCategory2", "SubCategory3", "SubCategory4", "SubCategory5"]

# Define the number of rows and partitions for data generation
partitions_requested = 2
data_rows = 10

# Define the schema for the synthetic data
schema = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("brand", StringType(), True),
    StructField("sub_brand", StringType(), True),
    StructField("category", StringType(), True),
    StructField("sub_category", StringType(), True),
    StructField("ean", StringType(), True),
    StructField("price", DoubleType(), True),  # Use DoubleType() for price
    StructField("created_at", TimestampType(), True),
    StructField("updated_at", TimestampType(), True)
])

# Create a DataGenerator for synthetic data generation
product_price_change_generator = (
    DataGenerator(spark, rows=data_rows, partitions=partitions_requested)
    .withSchema(schema)
    .withColumnSpec("item_id", minValue=1, maxValue=5000, step=1)
    .withColumnSpec("brand", text=PyfuncText(lambda context, v: random.choice(brand_values)))
    .withColumnSpec("sub_brand", text=PyfuncText(lambda context, v: random.choice(sub_brand_values)))
    .withColumnSpec("category", text=PyfuncText(lambda context, v: random.choice(category_values)))
    .withColumnSpec("sub_category", text=PyfuncText(lambda context, v: random.choice(sub_category_values)))
    .withColumnSpec("ean", text=PyfuncText(lambda context, v: Faker().ean8()))
    .withColumnSpec("price", random=True, text=PyfuncText(lambda context, v: str(round(random.uniform(100, 1000), 2))))
    .withColumnSpec("created_at", "timestamp", data_range=DateRange("2010-01-01 00:00:00", "2023-12-31 23:59:59", "days=1"), random=True)
    .withColumnSpec("updated_at", "timestamp", data_range=DateRange(lambda context, v: context['created_at'], "2023-12-31 23:59:59", "days=1"), random=True)
)

# Build the synthetic data DataFrame
product_price_change = product_price_change_generator.build()

# Display the first 10 rows of the generated data
display(product_price_change.limit(10))


---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<command-2246483205640141> in <cell line: 34>()
     42     .withColumnSpec("price", random=True, text=PyfuncText(lambda context, v: str(round(random.uniform(100, 1000), 2))))
     43     .withColumnSpec("created_at", "timestamp", data_range=DateRange("2010-01-01 00:00:00", "2023-12-31 23:59:59", "days=1"), random=True)
---> 44     .withColumnSpec("updated_at", "timestamp", data_range=DateRange(lambda context, v: context['created_at'], "2023-12-31 23:59:59", "days=1"), random=True)
     45 )
     46 

/local_disk0/.ephemeral_nfs/envs/pythonEnv-d1656ed8-0b5f-4b00-a1ee-0f51a1fec5f5/lib/python3.9/site-packages/dbldatagen/daterange.py in __init__(self, begin, end, interval, datetime_format)
     49         self.interval = interval if not isinstance(interval, str) else self._timedelta_from_string(interval)
     50 
---> 51         self.minVa

In [0]:
from dbldatagen import DataGenerator, PyfuncText, DateRange
from faker import Faker
import random
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType, DoubleType
from datetime import datetime, timedelta

# Create a SparkSession
spark = SparkSession.builder.appName("DataGeneration").getOrCreate()

brand_values = ["Brand1", "Brand2", "Brand3", "Brand4", "Brand5"]
sub_brand_values = ["SubBrand1", "SubBrand2", "SubBrand3", "SubBrand4", "SubBrand5"]
category_values = ["Category1", "Category2", "Category3", "Category4", "Category5"]
sub_category_values = ["SubCategory1", "SubCategory2", "SubCategory3", "SubCategory4", "SubCategory5"]

# Define the number of rows and partitions for data generation
partitions_requested = 2
data_rows = 10

# Define the schema for the synthetic data
schema = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("brand", StringType(), True),
    StructField("sub_brand", StringType(), True),
    StructField("category", StringType(), True),
    StructField("sub_category", StringType(), True),
    StructField("ean", StringType(), True),
    StructField("price", DoubleType(), True),  # Use DoubleType() for price
    StructField("created_at", TimestampType(), True),
    StructField("updated_at", TimestampType(), True)
])

# Custom function to generate updated_at >= created_at
def generate_updated_at(context, v):
    created_at = context["created_at"]
    max_days_difference = (datetime(2023, 12, 31) - created_at).days
    updated_at = created_at + timedelta(days=random.randint(0, max_days_difference))
    return updated_at

# Create a DataGenerator for synthetic data generation
item_data_generator = (
    DataGenerator(spark, rows=data_rows, partitions=partitions_requested)
    .withSchema(schema)
    .withColumnSpec("item_id", minValue=1, maxValue=5000, step=1)
    .withColumnSpec("brand", text=PyfuncText(lambda context, v: random.choice(brand_values)))
    .withColumnSpec("sub_brand", text=PyfuncText(lambda context, v: random.choice(sub_brand_values)))
    .withColumnSpec("category", text=PyfuncText(lambda context, v: random.choice(category_values)))
    .withColumnSpec("sub_category", text=PyfuncText(lambda context, v: random.choice(sub_category_values)))
    .withColumnSpec("ean", text=PyfuncText(lambda context, v: Faker().ean8()))
    .withColumnSpec("price", random=True, text=PyfuncText(lambda context, v: str(round(random.uniform(10, 1000), 2))))
    .withColumnSpec("created_at", "timestamp", data_range=DateRange("2010-01-01 00:00:00", "2023-12-31 23:59:59", "days=1"), random=True)
    .withColumnSpec("updated_at", "timestamp", data_range=DateRange("2023-01-01 00:00:00", "2023-12-31 23:59:59", custom_function=generate_updated_at), random=True)
)

# Build the synthetic data DataFrame
item_data = item_data_generator.build()

# Display the first 10 rows of the generated data
display(item_data.limit(10))


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<command-2246483205640142> in <cell line: 42>()
     50     .withColumnSpec("price", random=True, text=PyfuncText(lambda context, v: str(round(random.uniform(10, 1000), 2))))
     51     .withColumnSpec("created_at", "timestamp", data_range=DateRange("2010-01-01 00:00:00", "2023-12-31 23:59:59", "days=1"), random=True)
---> 52     .withColumnSpec("updated_at", "timestamp", data_range=DateRange("2023-01-01 00:00:00", "2023-12-31 23:59:59", custom_function=generate_updated_at), random=True)
     53 )
     54 

TypeError: __init__() got an unexpected keyword argument 'custom_function'

In [0]:
# Build the synthetic data DataFrame
item_data = item_data_generator.build()

# Display the first 10 rows of the generated data
item_data.show(10)


item_id,brand,sub_brand,category,sub_category,ean,price,created_at,updated_at
1,Brand1,SubBrand2,Category2,SubCategory3,81178353,0.0,2012-03-20T00:00:00.000+0000,2012-03-21
2,Brand3,SubBrand4,Category4,SubCategory4,96247877,1.0,2012-12-10T00:00:00.000+0000,2012-12-11
3,Brand2,SubBrand5,Category4,SubCategory1,64370170,2.0,2021-10-09T00:00:00.000+0000,2021-10-10
4,Brand5,SubBrand2,Category1,SubCategory1,48472609,3.0,2021-02-12T00:00:00.000+0000,2021-02-13
5,Brand3,SubBrand5,Category4,SubCategory3,18924374,4.0,2010-02-09T00:00:00.000+0000,2010-02-10
6,Brand4,SubBrand3,Category4,SubCategory4,87297454,5.0,2019-10-02T00:00:00.000+0000,2019-10-03
7,Brand5,SubBrand2,Category2,SubCategory1,54091450,6.0,2017-02-27T00:00:00.000+0000,2017-02-28
8,Brand4,SubBrand3,Category2,SubCategory5,09100107,7.0,2019-11-16T00:00:00.000+0000,2019-11-17
9,Brand1,SubBrand2,Category3,SubCategory3,95264400,8.0,2012-06-15T00:00:00.000+0000,2012-06-16
10,Brand4,SubBrand2,Category2,SubCategory5,82215255,9.0,2011-12-22T00:00:00.000+0000,2011-12-23
